In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
heart_data_original = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
heart_data = heart_data_original.copy()

In [ ]:
heart_data.rename(columns={'cp': 'chestpaintype',
                   'caa': '# major blood vessels',
                   'trtbps': 'resting blood pressure',
                   'chol': 'cholestrol',
                   'fbs': 'fasting blood sugar',
                   'restecg': 'resting electrocardiographic results',
                   'thalachh': 'max heart rate achieved',
                   'exng': 'exercise induced angina',
                   'oldpeak': 'ST Depression',
                   'slp': 'ST slope',
                   'thall': 'Thalassemia'
                  }, inplace=True)

In [ ]:
heart_data.head()

In [ ]:
heart_data.corr().style.background_gradient(cmap ='coolwarm')

In [ ]:
heart_data.describe().transpose()

### Chest Pain Type
    Value 0: typical angina
    Value 1: atypical angina
    Value 2: non-anginal pain
    Value 3: asymptomatic
   
### Output Type
    0 = less chance of heart attack 
    1= more chance of heart attack

In [ ]:
correlated_cols = ['sex', 'chestpaintype', 'exercise induced angina',
                   'ST slope', '# major blood vessels', 'Thalassemia']

for col in correlated_cols:
    heart_data.groupby(col).output.value_counts().unstack().plot(kind='bar')

### Building Model

In [ ]:
class PreProcessing:
    """Creates a trainable model"""
    def __init__(self, df):
        self.df = df
        
    def split_data(self):
        #Split data into train and test data
        train, test = train_test_split(self.df, test_size=0.2)
        return train, test
    
    def df_to_dataset(self, data, shuffle=True, batch_size=32):
        # Creates tf.data drom pandas daraframe
        data = data.copy()
        labels = data.pop('output')
        tensor_dataset = tf.data.Dataset.from_tensor_slices((dict(data), labels))
        if shuffle:
            tensor_dataset = tensor_dataset.shuffle(buffer_size=len(data))
        tensor_dataset = tensor_dataset.batch(batch_size)
        tensor_dataset = tensor_dataset.prefetch(batch_size)
        return tensor_dataset
    
    def get_normalization_layer(self, name, dataset):
        # Normalizes the input to have 0 mean and 1 standard deviation
        normalizer = preprocessing.Normalization()
        feature_ds = dataset.map(lambda x, y: x[name])
        normalizer.adapt(feature_ds)
        return normalizer
    
    def get_normalized_features(self):
        train, test = self.split_data()
        train_ds = self.df_to_dataset(train, batch_size=4)
        test_ds = self.df_to_dataset(test, batch_size=4)
        all_inputs = []
        encoded_features = []
        for header in self.df:
            if header != 'output':
                numeric_col = tf.keras.Input(shape=(1,), name=header)
                normalization_layer = self.get_normalization_layer(header, train_ds)
                encoded_numeric_col = normalization_layer(numeric_col)
                all_inputs.append(numeric_col)
                encoded_features.append(encoded_numeric_col)
        return all_inputs, encoded_features, train_ds, test_ds
        
    

In [ ]:
preProcessing = PreProcessing(heart_data)
all_inputs, encoded_features, train_ds, test_ds = preProcessing.get_normalized_features()

In [ ]:
class BuildModel:
    """Creates model for prediction"""
    def __init__(self, all_inputs, encoded_features):
        self.all_inputs = all_inputs
        self.encoded_features = encoded_features
    
    def create_model(self):
        all_features = tf.keras.layers.concatenate(encoded_features)
        x = tf.keras.layers.Dense(32, activation='relu')(all_features)
        x = tf.keras.layers.Dropout(0.5)(x)
        output = tf.keras.layers.Dense(1)(x)
        self.model = tf.keras.Model(self.all_inputs, output)
        self.model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])
        return None
    
    def fit(self, train_ds, valid_dataset, epochs=10):
        self.create_model()
        self.model.fit(train_ds, epochs=epochs, validation_data = valid_dataset)
        return None
        

In [ ]:
buildModel = BuildModel(all_inputs, encoded_features)
buildModel.fit(train_ds, test_ds)

In [ ]:
tf.keras.utils.plot_model(buildModel.model, show_shapes=True, rankdir="LR")